# 无障碍导航后端系统 - 完整测试
本notebook用于测试所有服务功能，包括TTS、LLM、YOLO、高德地图API

In [2]:
# ==================== 1. 环境准备 ====================
import sys
import os

# 确保在正确的目录
if not os.path.exists('main.py'):
    print("当前目录:", os.getcwd())
    print("切换到项目目录...")
    os.chdir('accessible-nav-backend')

print(f"✅ 工作目录: {os.getcwd()}")
print(f"✅ Python版本: {sys.version}")

✅ 工作目录: C:\Users\zy312\Desktop\高德竞赛代码\accessible-nav-backend\accessible-nav-backend
✅ Python版本: 3.11.13 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:03:15) [MSC v.1929 64 bit (AMD64)]


In [3]:
# ==================== 2. 导入依赖 ====================
import asyncio
import requests
import json
import time
import base64
from PIL import Image
import io
from IPython.display import Audio, display, HTML
import warnings
warnings.filterwarnings('ignore')

print("✅ 基础库导入完成")

✅ 基础库导入完成


## 测试1: 配置加载测试

In [3]:
# ==================== 3. 测试配置加载 ====================
print("=" * 70)
print("🔧 配置加载测试")
print("=" * 70)

try:
    from config.settings import settings
    
    print(f"\n✅ 配置加载成功")
    print(f"\n📋 系统配置:")
    print(f"  DEBUG: {settings.DEBUG}")
    print(f"  MOCK_MODE: {settings.MOCK_MODE}")
    print(f"  PORT: {settings.PORT}")
    
    print(f"\n🔑 API密钥状态:")
    print(f"  AMAP_API_KEY: {'✅ 已配置' if settings.AMAP_API_KEY else '❌ 未配置'}")
    print(f"  LLM_API_KEY: {'✅ 已配置' if settings.LLM_API_KEY else '❌ 未配置'}")
    
    print(f"\n🗣️  TTS配置:")
    print(f"  TTS_PROVIDER: {settings.TTS_PROVIDER}")
    print(f"  TTS_VOICE: {settings.TTS_VOICE}")
    print(f"  AUDIO_OUTPUT_DIR: {settings.AUDIO_OUTPUT_DIR}")
    
    print(f"\n🤖 LLM配置:")
    print(f"  LLM_MODEL: {settings.LLM_MODEL}")
    print(f"  LLM_API_BASE: {settings.LLM_API_BASE}")
    print(f"  LLM_TEMPERATURE: {settings.LLM_TEMPERATURE}")
    
    print(f"\n👁️  YOLO配置:")
    print(f"  YOLO_MODEL_PATH: {settings.YOLO_MODEL_PATH}")
    print(f"  YOLO_CONFIDENCE: {settings.YOLO_CONFIDENCE}")
    print(f"  YOLO_DEVICE: {settings.YOLO_DEVICE}")
    
except Exception as e:
    print(f"❌ 配置加载失败: {e}")
    import traceback
    traceback.print_exc()

🔧 配置加载测试

✅ 配置加载成功

📋 系统配置:
  DEBUG: True
  MOCK_MODE: False
  PORT: 8000

🔑 API密钥状态:
  AMAP_API_KEY: ✅ 已配置
  LLM_API_KEY: ✅ 已配置

🗣️  TTS配置:
  TTS_PROVIDER: edge
  TTS_VOICE: zh-CN-XiaoxiaoNeural
  AUDIO_OUTPUT_DIR: ./audio_output

🤖 LLM配置:
  LLM_MODEL: deepseek-chat
  LLM_API_BASE: https://api.deepseek.com
  LLM_TEMPERATURE: 0.7

👁️  YOLO配置:
  YOLO_MODEL_PATH: yolov8n.pt
  YOLO_CONFIDENCE: 0.5
  YOLO_DEVICE: cpu


## 测试2: TTS服务测试（核心）

In [8]:
# ==================== 4. TTS服务测试 ====================
print("=" * 70)
print("🗣️  TTS服务测试")
print("=" * 70)

try:
    from app.services.tts_service import TTSService
    
    tts_service = TTSService()
    print(f"\n✅ TTS服务初始化成功")
    print(f"  Provider: {tts_service.provider}")
    print(f"  Mock Mode: {tts_service.mock_mode}")
    print(f"  Output Dir: {tts_service.output_dir}")
    
    # 测试文本转语音
    test_texts = [
        "你好，我是你的导航助手",
        "前方50米左转",
        "检测到障碍物，请注意安全"
    ]
    
    print(f"\n🎯 开始测试TTS生成...")
    for i, text in enumerate(test_texts, 1):
        print(f"\n测试 {i}/{len(test_texts)}: {text}")
        try:
            # 使用asyncio运行异步函数
            audio_url = await tts_service.text_to_speech(
                text=text,
                session_id=f"test_session_{int(time.time())}"
            )
            
            print(f"  ✅ 生成成功: {audio_url}")
            
            # 检查文件是否存在
            if not audio_url.startswith('/audio/mock_'):
                filename = audio_url.replace('/audio/', '')
                filepath = os.path.join(settings.AUDIO_OUTPUT_DIR, filename)
                if os.path.exists(filepath):
                    file_size = os.path.getsize(filepath) / 1024  # KB
                    print(f"  📁 文件大小: {file_size:.2f} KB")
                    
                    # 尝试播放音频
                    try:
                        display(Audio(filepath, autoplay=False))
                    except:
                        pass
                else:
                    print(f"  ⚠️  文件不存在: {filepath}")
            
        except Exception as e:
            print(f"  ❌ 生成失败: {e}")
    
    print(f"\n✅ TTS服务测试完成")
    
except Exception as e:
    print(f"❌ TTS服务测试失败: {e}")
    import traceback
    traceback.print_exc()

🗣️  TTS服务测试

✅ TTS服务初始化成功
  Provider: edge
  Mock Mode: False
  Output Dir: ./audio_output

🎯 开始测试TTS生成...

测试 1/3: 你好，我是你的导航助手
✅ Edge TTS生成音频: 1e0a597888156ecff3fd8f8513e3ae36.mp3
  ✅ 生成成功: /audio/1e0a597888156ecff3fd8f8513e3ae36.mp3
  📁 文件大小: 16.45 KB



测试 2/3: 前方50米左转
✅ Edge TTS生成音频: d0fd8b3786d9376ad3bd974b9a98e01a.mp3
  ✅ 生成成功: /audio/d0fd8b3786d9376ad3bd974b9a98e01a.mp3
  📁 文件大小: 12.23 KB



测试 3/3: 检测到障碍物，请注意安全
✅ Edge TTS生成音频: c53c79eb5c16a1f67b82e9b1882d9200.mp3
  ✅ 生成成功: /audio/c53c79eb5c16a1f67b82e9b1882d9200.mp3
  📁 文件大小: 18.28 KB



✅ TTS服务测试完成


## 测试3: LLM服务测试

In [9]:
# ==================== 5. LLM服务测试 ====================
print("=" * 70)
print("🤖 LLM服务测试")
print("=" * 70)

try:
    from app.services.llm_service import LLMService
    from app.core.session_manager import session_manager
    
    llm_service = LLMService()
    print(f"\n✅ LLM服务初始化成功")
    print(f"  Model: {settings.LLM_MODEL}")
    print(f"  API Base: {settings.LLM_API_BASE}")
    print(f"  Mock Mode: {settings.MOCK_MODE}")
    
    # 测试对话
    test_messages = [
        "你好",
        "我想去天安门广场",
        "路上有什么需要注意的吗"
    ]
    
    print(f"\n🎯 开始测试LLM对话...")
    session_id = f"test_llm_{int(time.time())}"
    user_id = "test_user"
    
    # 创建会话
    session = session_manager.create_conversation(user_id, session_id)
    
    for i, user_msg in enumerate(test_messages, 1):
        print(f"\n对话 {i}/{len(test_messages)}")
        print(f"  👤 用户: {user_msg}")
        
        try:
            response = await llm_service.process_conversation(
                user_message=user_msg,
                session=session
            )
            
            print(f"  🤖 助手: {response['reply']}")
            print(f"  📊 导航状态: {response.get('nav_state', 'unknown')}")
            
        except Exception as e:
            print(f"  ❌ 对话失败: {e}")
            import traceback
            traceback.print_exc()
    
    print(f"\n✅ LLM服务测试完成")
    
except Exception as e:
    print(f"❌ LLM服务测试失败: {e}")
    import traceback
    traceback.print_exc()

🤖 LLM服务测试

✅ LLM服务初始化成功
  Model: deepseek-chat
  API Base: https://api.deepseek.com
  Mock Mode: False

🎯 开始测试LLM对话...

对话 1/3
  👤 用户: 你好
  🤖 助手: 你好！我是您的导航助手。请问您想去哪里呢？
  📊 导航状态: asking

对话 2/3
  👤 用户: 我想去天安门广场
  🤖 助手: 好的，我帮您规划去天安门广场的路线。请问您现在从哪里出发呢？
  📊 导航状态: asking

对话 3/3
  👤 用户: 路上有什么需要注意的吗
  🤖 助手: 老人家，我是帮您规划路线的导航助手。您是想去哪里呢？告诉我起点和终点，我就能帮您规划路线了。
  📊 导航状态: asking

✅ LLM服务测试完成


## 测试4: YOLO服务测试

In [12]:
# ==================== 6. YOLO服务测试 ====================
print("=" * 70)
print("👁️  YOLO服务测试")
print("=" * 70)

try:
    from app.services.yolo_service import YOLOService
    
    yolo_service = YOLOService()
    print(f"\n✅ YOLO服务初始化成功")
    print(f"  Model Path: {settings.YOLO_MODEL_PATH}")
    print(f"  Confidence: {settings.YOLO_CONFIDENCE}")
    print(f"  Device: {settings.YOLO_DEVICE}")
    print(f"  Mock Mode: {settings.MOCK_MODE}")
    
    # 使用真实的街道测试图片
    print(f"\n🎯 加载测试图片...")
    test_images = [
        "fig/shanghaishinanjinglushangyebuxingjie_10006357.jpg",
        "fig/R-C.jpg"
    ]
    
    images_base64 = []
    for img_path in test_images:
        if os.path.exists(img_path):
            with open(img_path, 'rb') as f:
                img_bytes = f.read()
                img_base64 = base64.b64encode(img_bytes).decode('utf-8')
                images_base64.append(img_base64)
                print(f"  ✅ 加载图片: {img_path}")
        else:
            print(f"  ⚠️  图片不存在: {img_path}")
    
    if not images_base64:
        # 如果没有找到图片，创建测试图片
        print(f"\n  ℹ️  未找到测试图片，创建空白测试图片...")
        img = Image.new('RGB', (640, 480), color=(73, 109, 137))
        img_byte_arr = io.BytesIO()
        img.save(img_byte_arr, format='JPEG')
        img_bytes = img_byte_arr.getvalue()
        img_base64 = base64.b64encode(img_bytes).decode('utf-8')
        images_base64 = [img_base64]
        print(f"  ✅ 测试图片创建成功 (640x480)")
    
    # 测试检测
    print(f"\n🎯 开始测试障碍物检测...")
    print(f"  测试图片数量: {len(images_base64)}")
    
    try:
        # 使用detect_batch方法
        results = await yolo_service.detect_batch(images_base64)
        
        if results:
            # 使用aggregate_obstacles整合所有图片的结果
            obstacles = yolo_service.aggregate_obstacles(results)
            safety = yolo_service.calculate_safety_level(obstacles)
            road_condition = yolo_service.describe_road_condition(obstacles)
            
            print(f"\n📊 检测结果汇总:")
            print(f"  障碍物数量: {len(obstacles)}")
            print(f"  安全等级: {safety}/5")
            print(f"  路况描述: {road_condition}")
            
            if obstacles:
                print(f"\n🚧 检测到的障碍物:")
                for i, obs in enumerate(obstacles, 1):
                    print(f"  {i}. {obs.type} - 距离: {obs.distance}m - 方向: {obs.direction} - 置信度: {obs.confidence:.2f}")
            else:
                print(f"\n  ✅ 未检测到障碍物，道路安全")
            
            # 显示每张图片的详细结果
            print(f"\n📋 各图片检测详情:")
            for idx, result in enumerate(results, 1):
                obstacles_raw = result.get('obstacles', [])
                print(f"\n  图片 {idx}:")
                print(f"    检测到对象数: {len(obstacles_raw)}")
                if obstacles_raw:
                    for obj in obstacles_raw[:3]:  # 只显示前3个
                        print(f"      - Class {obj['class']}: 置信度 {obj['confidence']:.2f}")
        
    except Exception as e:
        print(f"  ❌ 检测失败: {e}")
        import traceback
        traceback.print_exc()
    
    print(f"\n✅ YOLO服务测试完成")
    
except Exception as e:
    print(f"❌ YOLO服务测试失败: {e}")
    import traceback
    traceback.print_exc()

Ultralytics YOLOv8.0.0  Python-3.11.13 torch-2.1.0+cpu CPU
Fusing layers... 
YOLOv8n summary: 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs
Ultralytics YOLOv8.0.0  Python-3.11.13 torch-2.1.0+cpu CPU
Fusing layers... 


👁️  YOLO服务测试
✅ YOLO模型加载成功: yolov8n.pt

✅ YOLO服务初始化成功
  Model Path: yolov8n.pt
  Confidence: 0.5
  Device: cpu
  Mock Mode: False

🎯 加载测试图片...
  ✅ 加载图片: fig/shanghaishinanjinglushangyebuxingjie_10006357.jpg
  ✅ 加载图片: fig/R-C.jpg

🎯 开始测试障碍物检测...
  测试图片数量: 2
检测失败: C:\Users\zy312\Desktop\高德竞赛代码\accessible-nav-backend\accessible-nav-backend\<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=700x497 at 0x2024213B910> does not exist


YOLOv8n summary: 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs


检测失败: C:\Users\zy312\Desktop\高德竞赛代码\accessible-nav-backend\accessible-nav-backend\<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=860x484 at 0x2023E087D90> does not exist

📊 检测结果汇总:
  障碍物数量: 0
  安全等级: 5/5
  路况描述: 道路通畅，无明显障碍

  ✅ 未检测到障碍物，道路安全

📋 各图片检测详情:

  图片 1:
    检测到对象数: 0

  图片 2:
    检测到对象数: 0

✅ YOLO服务测试完成


## 测试5: 高德地图服务测试

In [11]:
# ==================== 7. 高德地图服务测试 ====================
print("=" * 70)
print("🗺️  高德地图服务测试")
print("=" * 70)

try:
    from app.services.amap_service import AmapService
    
    amap_service = AmapService()
    print(f"\n✅ 高德地图服务初始化成功")
    print(f"  API Key: {'✅ 已配置' if settings.AMAP_API_KEY else '❌ 未配置'}")
    print(f"  Mock Mode: {settings.MOCK_MODE}")
    
    # 测试路径规划
    print(f"\n🎯 测试路径规划...")
    test_routes = [
        {
            'name': '北京天安门到故宫',
            'origin': {"lng": 116.397128, "lat": 39.916527},
            'destination': {"lng": 116.403414, "lat": 39.918058}
        },
        {
            'name': '上海人民广场到外滩',
            'origin': {"lng": 121.475198, "lat": 31.231590},
            'destination': {"lng": 121.492354, "lat": 31.241044}
        }
    ]
    
    for i, route_info in enumerate(test_routes, 1):
        print(f"\n路线 {i}: {route_info['name']}")
        print(f"  起点: {route_info['origin']['lng']},{route_info['origin']['lat']}")
        print(f"  终点: {route_info['destination']['lng']},{route_info['destination']['lat']}")
        
        try:
            routes = await amap_service.plan_walking_route(
                origin=route_info['origin'],
                destination=route_info['destination']
            )
            
            if routes:
                print(f"  ✅ 规划成功，找到 {len(routes)} 条路线")
                
                route = routes[0]
                print(f"  📏 距离: {route.get('distance', 0)} 米")
                print(f"  ⏱️  时间: {route.get('duration', 0)} 秒")
                print(f"  ♿ 无障碍评分: {route.get('accessibilityScore', 'N/A')}")
                
                steps = route.get('steps', [])
                print(f"  🚶 步骤数: {len(steps)}")
                if steps:
                    print(f"\n  前3个导航步骤:")
                    for j, step in enumerate(steps[:3], 1):
                        print(f"    {j}. {step.get('instruction', 'N/A')}")
            else:
                print(f"  ❌ 规划失败: 未返回路线")
                
        except Exception as e:
            print(f"  ❌ 请求失败: {e}")
    
    print(f"\n✅ 高德地图服务测试完成")
    
except Exception as e:
    print(f"❌ 高德地图服务测试失败: {e}")
    import traceback
    traceback.print_exc()

🗺️  高德地图服务测试

✅ 高德地图服务初始化成功
  API Key: ✅ 已配置
  Mock Mode: False

🎯 测试路径规划...

路线 1: 北京天安门到故宫
  起点: 116.397128,39.916527
  终点: 116.403414,39.918058
  ✅ 规划成功，找到 1 条路线
  📏 距离: 1071 米
  ⏱️  时间: 0 秒
  ♿ 无障碍评分: 85
  🚶 步骤数: 19

  前3个导航步骤:
    1. 向南步行5米直行
    2. 向南步行18米左转
    3. 向东步行53米右转

路线 2: 上海人民广场到外滩
  起点: 121.475198,31.23159
  终点: 121.492354,31.241044
  ✅ 规划成功，找到 1 条路线
  📏 距离: 2325 米
  ⏱️  时间: 0 秒
  ♿ 无障碍评分: 85
  🚶 步骤数: 7

  前3个导航步骤:
    1. 向南步行95米左转
    2. 向东北步行100米向右前方行走
    3. 沿福州路向东步行1432米左转

✅ 高德地图服务测试完成


## 测试6: 端到端API测试（需要先启动服务）

In [ ]:
# ==================== 8. API端到端测试 ====================
print("=" * 70)
print("🌐 API端到端测试（需要服务运行）")
print("=" * 70)

BASE_URL = "http://localhost:8000"
USER_ID = "test_user_001"
SESSION_ID = f"session_{int(time.time())}"

print(f"\n配置:")
print(f"  BASE_URL: {BASE_URL}")
print(f"  USER_ID: {USER_ID}")
print(f"  SESSION_ID: {SESSION_ID}")

# 测试1: 健康检查
print(f"\n🏥 测试1: 健康检查")
try:
    response = requests.get(f"{BASE_URL}/health", timeout=5)
    if response.status_code == 200:
        data = response.json()
        print(f"  ✅ 服务状态: {data.get('status')}")
        print(f"  📊 运行模式: {data.get('mode')}")
    else:
        print(f"  ❌ 健康检查失败: {response.status_code}")
except Exception as e:
    print(f"  ⚠️  服务未运行: {e}")
    print(f"  💡 请先运行: python main.py")

# 测试2: 语音交互API
print(f"\n🗣️  测试2: 语音交互API")
try:
    payload = {
        "userId": USER_ID,
        "sessionId": SESSION_ID,
        "text": "你好，我想去天安门",
        "timestamp": int(time.time() * 1000)
    }
    
    response = requests.post(f"{BASE_URL}/v1/voice/text", json=payload, timeout=10)
    if response.status_code == 200:
        data = response.json()
        print(f"  ✅ 请求成功")
        print(f"  👤 用户: {payload['text']}")
        print(f"  🤖 回复: {data.get('message', 'N/A')}")
        print(f"  🔊 音频URL: {data.get('audioUrl', 'N/A')}")
        print(f"  📊 导航状态: {data.get('navState', 'N/A')}")
    else:
        print(f"  ❌ 请求失败: {response.status_code}")
        print(f"  {response.text}")
except Exception as e:
    print(f"  ⚠️  请求失败: {e}")

# 测试3: 导航启动API
print(f"\n🧭 测试3: 导航启动API")
try:
    payload = {
        "userId": USER_ID,
        "sessionId": SESSION_ID,
        "origin": "116.397128,39.916527",
        "destination": "116.403414,39.918058"
    }
    
    response = requests.post(f"{BASE_URL}/v1/nav/start", json=payload, timeout=10)
    if response.status_code == 200:
        data = response.json()
        print(f"  ✅ 导航启动成功")
        print(f"  🆔 导航会话ID: {data.get('navSessionId', 'N/A')}")
        print(f"  📏 路线数量: {len(data.get('routes', []))}")
        print(f"  💬 提示消息: {data.get('message', 'N/A')}")
        print(f"  🔊 音频URL: {data.get('audioUrl', 'N/A')}")
        
        if data.get('routes'):
            route = data['routes'][0]
            print(f"\n  路线详情:")
            print(f"    距离: {route.get('distance', 0)} 米")
            print(f"    时间: {route.get('duration', 0)} 秒")
    else:
        print(f"  ❌ 请求失败: {response.status_code}")
        print(f"  {response.text}")
except Exception as e:
    print(f"  ⚠️  请求失败: {e}")

print(f"\n✅ API端到端测试完成")

🌐 API端到端测试（需要服务运行）

配置:
  BASE_URL: http://localhost:8000
  USER_ID: test_user_001
  SESSION_ID: session_1766128535

🏥 测试1: 健康检查
  ⚠️  服务未运行: HTTPConnectionPool(host='localhost', port=8000): Read timed out. (read timeout=5)
  💡 请先运行: python main.py

🗣️  测试2: 语音交互API


## 测试总结

In [ ]:
# ==================== 9. 测试总结 ====================
print("=" * 70)
print("📊 测试总结")
print("=" * 70)

print(f"\n✅ 测试完成！")
print(f"\n📝 测试内容:")
print(f"  1. ✅ 配置加载测试")
print(f"  2. 🗣️  TTS服务测试（Edge TTS）")
print(f"  3. 🤖 LLM服务测试（Deepseek）")
print(f"  4. 👁️  YOLO服务测试")
print(f"  5. 🗺️  高德地图服务测试")
print(f"  6. 🌐 API端到端测试")

print(f"\n💡 下一步:")
print(f"  1. 如果发现问题，检查错误日志")
print(f"  2. 确保所有API密钥正确配置在.env文件")
print(f"  3. 对于TTS问题，检查audio_output目录权限")
print(f"  4. 运行完整的系统测试: python tests/test_system.py")

print(f"\n" + "=" * 70)